In [104]:
import os 
os.environ['PROJ_LIB'] = "/home/jesseake/skagit-met/.pixi/envs/analysis/share/proj"
import xarray as xr

In [105]:
import dask
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import holoviews as hv
import datetime as dt
import hvplot.xarray
import rioxarray as rxr
import geopandas as gpd
from shapely import vectorized
hv.extension('bokeh')

In [3]:
grid_path = '/data0/skagit_met/PNNL/historical/SERDP6km.geo_em.d01.nc'
ds_grid = xr.open_dataset(grid_path).squeeze() # Drop Time=0 scalar dimension
# Assign Coordinates in Xarray (keep landmask for viz) 
dsg = ds_grid[['LANDMASK','CLONG','CLAT']]
# Still not recognized as multidimensional coordinates
# Rename to x and y To match data files
dsg = dsg.set_coords(("CLONG", "CLAT")).rename(dict(south_north='y', west_east='x'))
dsg


<xarray.Dataset> Size: 2MB
Dimensions:   (y: 450, x: 450)
Coordinates:
    CLONG     (y, x) float32 810kB ...
    CLAT      (y, x) float32 810kB ...
Dimensions without coordinates: y, x
Data variables:
    LANDMASK  (y, x) float32 810kB ...
Attributes: (12/47)
    TITLE:                           OUTPUT FROM GEOGRID V3.8
    SIMULATION_START_DATE:           0000-00-00_00:00:00
    WEST-EAST_GRID_DIMENSION:        451
    SOUTH-NORTH_GRID_DIMENSION:      451
    BOTTOM-TOP_GRID_DIMENSION:       0
    WEST-EAST_PATCH_START_UNSTAG:    1
    ...                              ...
    parent_grid_ratio:               1
    sr_x:                            1
    sr_y:                            1
    FLAG_MF_XY:                      1
    FLAG_LAI12M:                     1
    FLAG_LAKE_DEPTH:                 1

In [4]:
virtual_pnnl = '/data0/skagit_met/PNNL/historical/PNNL_historical.parquet'
pnnl = xr.open_dataset(virtual_pnnl, engine='kerchunk', mask_and_scale=False)
pnnl.assign_coords(dsg.coords)
# Also bring in land mask
pnnl.coords['LANDMASK'] = dsg.LANDMASK
skagit = gpd.read_file('~/skagit-met/Data/GIS/SkagitBoundary.json')
mask = vectorized.contains(skagit.geometry[0], pnnl.CLONG.values, pnnl.CLAT.values)
pnnl = pnnl.where(mask)
# pnnl = pnnl.rio.write_crs("EPSG:4326", inplace=True)
# pnnl = pnnl.rio.clip(skagit.to_crs(pnnl.rio.crs).geometry)

ornl_2003 = xr.open_zarr('/data0/skagit_met/atmospheric_rivers/2003_2003_ORNL_data.zarr')
ornl_2003 = ornl_2003.sel(time=slice('2003-10-14','2003-10-28'))
PRSIM_2003 = xr.open_zarr('/data0/skagit_met/atmospheric_rivers/2003-10-14_2003-10-28_PRISM_data.zarr')
wrf_2003 = xr.open_zarr('/data0/skagit_met/atmospheric_rivers/2003-10-14_2003-10-28_wrf_era5_data.zarr')
snotel_2003 = xr.open_zarr('/data0/skagit_met/atmospheric_rivers/2003-10-14_2003-10-28_SNOTEL_daily_data.zarr')
pnnl_2003 = pnnl.sel(time=slice('2003-10-14', '2003-10-28'))

ornl_2006 = xr.open_zarr('/data0/skagit_met/atmospheric_rivers/2006_2006_ORNL_data.zarr')
ornl_2006 = ornl_2006.sel(time=slice('2006-10-31','2006-11-13'))
PRSIM_2006 = xr.open_zarr('/data0/skagit_met/atmospheric_rivers/2006-10-31_2006-11-13_PRISM_data.zarr')
wrf_2006 = xr.open_zarr('/data0/skagit_met/atmospheric_rivers/2006-10-31_2006-11-13_wrf_era5_data.zarr')
pnnl_2006 = pnnl.sel(time=slice('2006-10-31','2006-11-13'))
snotel_2006 = xr.open_zarr('/data0/skagit_met/atmospheric_rivers/2006-10-31_2006-11-13_SNOTEL_daily_data.zarr')

# Load the same datsets for 1995 and 2003 and 1990 and compare the data
ornl_1995 = xr.open_zarr('/data0/skagit_met/atmospheric_rivers/1995_1995_ORNL_data.zarr')
ornl_1995 = ornl_1995.sel(time=slice('1995-11-22','1995-12-06'))
PRSIM_1995 = xr.open_zarr('/data0/skagit_met/atmospheric_rivers/1995-11-22_1995-12-06_PRISM_data.zarr')
wrf_1995 = xr.open_zarr('/data0/skagit_met/atmospheric_rivers/1995-11-22_1995-12-06_wrf_era5_data.zarr')
pnnl_1995 = pnnl.sel(time=slice('1995-11-22','1995-12-06'))
# snotel_1995 = xarray.open_zarr('/data0/skagit_met/atmospheric_rivers/1995-11-22_1995-12-06_SNOTEL_daily_data.zarr')

ornl_1990 = xr.open_zarr('/data0/skagit_met/atmospheric_rivers/1990_1990_ORNL_data.zarr')
ornl_1990 = ornl_1990.sel(time=slice('1990-11-03','1990-11-17'))
PRSIM_1990 = xr.open_zarr('/data0/skagit_met/atmospheric_rivers/1990-11-03_1990-11-17_PRISM_data.zarr')
wrf_1990 = xr.open_zarr('/data0/skagit_met/atmospheric_rivers/1990-11-03_1990-11-17_wrf_era5_data.zarr')
pnnl_1990 = pnnl.sel(time=slice('1990-11-03','1990-11-17'))
snotel_1990 = xr.open_zarr('/data0/skagit_met/atmospheric_rivers/1990-11-03_1990-11-17_SNOTEL_daily_data.zarr')

In [5]:
wrf_1990['T2C'] = wrf_1990['T2'] - 273.15
wrf_1990['PRCP'] = (wrf_1990['RAINC']+ wrf_1990['RAINNC'])
pnnl_1990['T2C'] = pnnl_1990['T2'] - 273.15

ornl_1990['tmean'] = (ornl_1990.tmax + ornl_1990.tmin) / 2
wrf_1990_daily = wrf_1990.resample(time='1D').mean()
pnnl_1990_daily = pnnl_1990.resample(time='1D').mean()

wrf_1995['T2C'] = wrf_1995['T2'] - 273.15
wrf_1995['PRCP'] = (wrf_1995['RAINC']+ wrf_1995['RAINNC'])
pnnl_1995['T2C'] = pnnl_1995['T2'] - 273.15

ornl_1995['tmean'] = (ornl_1995.tmax + ornl_1995.tmin) / 2
wrf_1995_daily = wrf_1995.resample(time='1D').mean()
pnnl_1995_daily = pnnl_1995.resample(time='1D').mean()

wrf_2003['T2C'] = wrf_2003['T2'] - 273.15
wrf_2003['PRCP'] = (wrf_2003['RAINC']+ wrf_2003['RAINNC'])
pnnl_2003['T2C'] = pnnl_2003['T2'] - 273.15

ornl_2003['tmean'] = (ornl_2003.tmax + ornl_2003.tmin) / 2
wrf_2003_daily = wrf_2003.resample(time='1D').mean()
pnnl_2003_daily = pnnl_2003.resample(time='1D').mean()

wrf_2006['T2C'] = wrf_2006['T2'] - 273.15
wrf_2006['PRCP'] = (wrf_2006['RAINC']+ wrf_2006['RAINNC'])
pnnl_2006['T2C'] = pnnl_2006['T2'] - 273.15

ornl_2006['tmean'] = (ornl_2006.tmax + ornl_2006.tmin) / 2
wrf_2006_daily = wrf_2006.resample(time='1D').mean()
pnnl_2006_daily = pnnl_2006.resample(time='1D').mean()

In [6]:
# ornl_1990_ds = hv.Dataset(ornl_1990.mean(('lon','lat')), kdims=['time'], vdims=['tmax', 'tmin', 'tmean', 'lrad', 'prcp', 'srad', 'rhum', 'wind'])
# ornl_1995_ds = hv.Dataset(ornl_1995.mean(('lon','lat')), kdims=['time'], vdims=['tmax', 'tmin', 'tmean', 'lrad', 'prcp', 'srad', 'rhum', 'wind'])
# ornl_2003_ds = hv.Dataset(ornl_2003.mean(('lon','lat')), kdims=['time'], vdims=['tmax', 'tmin', 'tmean', 'lrad', 'prcp', 'srad', 'rhum', 'wind'])
# ornl_2006_ds = hv.Dataset(ornl_2006.mean(('lon','lat')), kdims=['time'], vdims=['tmax', 'tmin', 'tmean', 'lrad', 'prcp', 'srad', 'rhum', 'wind'])

# prism_1990_ds = hv.Dataset(PRSIM_1990.shift(time=-1).mean(('lon','lat')))
# prism_1995_ds = hv.Dataset(PRSIM_1995.shift(time=-1).mean(('lon','lat')))
# prism_2003_ds = hv.Dataset(PRSIM_2003.shift(time=-1).mean(('lon','lat')))
# prism_2006_ds = hv.Dataset(PRSIM_2006.shift(time=-1).mean(('lon','lat')))

# wrf_1990_ds_mean = hv.Dataset(wrf_1990_daily.mean(('y', 'x')))
# wrf_1995_ds_mean = hv.Dataset(wrf_1995_daily.mean(('y', 'x')))
# wrf_2003_ds_mean = hv.Dataset(wrf_2003_daily.mean(('y', 'x')))
# wrf_2006_ds_mean = hv.Dataset(wrf_2006_daily.mean(('y', 'x')))

# wrf_1990_ds_max = hv.Dataset(wrf_1990_daily.max(('y', 'x')))
# wrf_1995_ds_max = hv.Dataset(wrf_1995_daily.max(('y', 'x')))
# wrf_2003_ds_max = hv.Dataset(wrf_2003_daily.max(('y', 'x')))
# wrf_2006_ds_max = hv.Dataset(wrf_2006_daily.max(('y', 'x')))

# wrf_1990_ds_min = hv.Dataset(wrf_1990_daily.max(('y', 'x')))
# wrf_1995_ds_min = hv.Dataset(wrf_1995_daily.max(('y', 'x')))
# wrf_2003_ds_min = hv.Dataset(wrf_2003_daily.max(('y', 'x')))
# wrf_2006_ds_min = hv.Dataset(wrf_2006_daily.max(('y', 'x')))

# pnnl_1990_ds_mean = hv.Dataset(pnnl_1990_daily.mean(('x','y')))
# pnnl_1995_ds_mean = hv.Dataset(pnnl_1995_daily.mean(('y', 'x')))
# pnnl_2003_ds_mean = hv.Dataset(pnnl_2003_daily.mean(('y', 'x')))
# pnnl_2006_ds_mean = hv.Dataset(pnnl_2006_daily.mean(('y', 'x')))

# pnnl_1990_ds_max = hv.Dataset(pnnl_1990_daily.max(('y', 'x')))
# pnnl_1995_ds_max = hv.Dataset(pnnl_1995_daily.max(('y', 'x')))
# pnnl_2003_ds_max = hv.Dataset(pnnl_2003_daily.max(('y', 'x')))
# pnnl_2006_ds_max = hv.Dataset(pnnl_2006_daily.max(('y', 'x')))

# pnnl_1990_ds_min = hv.Dataset(pnnl_1990_daily.min(('y', 'x')))
# pnnl_1995_ds_min = hv.Dataset(pnnl_1990_daily.min(('y', 'x')))
# pnnl_2003_ds_min = hv.Dataset(pnnl_1990_daily.min(('y', 'x')))
# pnnl_2006_ds_min = hv.Dataset(pnnl_1990_daily.min(('y', 'x')))

In [7]:
# Let's plot mean temperature for the 4 datasets
# ornl_1990_temp_mean = ornl_1990_ds.to(hv.Curve, kdims=['time'], vdims=['tmean'], group='Temperature', label='ORNL (Daymet) 1990').opts(tools=['hover'])
# ornl_1995_temp_mean = ornl_1995_ds.to(hv.Curve, kdims=['time'], vdims=['tmean'], group='Temperature', label='ORNL (Daymet) 1995').opts(tools=['hover'])
# ornl_2003_temp_mean = ornl_2003_ds.to(hv.Curve, kdims=['time'], vdims=['tmean'], group='Temperature', label='ORNL (Daymet) 2003').opts(tools=['hover'])
# ornl_2006_temp_mean = ornl_2006_ds.to(hv.Curve, kdims=['time'], vdims=['tmean'], group='Temperature', label='ORNL (Daymet) 2006').opts(tools=['hover'])

# prism_1990_temp_mean  = prism_1990_ds.to(hv.Curve, kdims=['time'], vdims=['tmean'], group='Temperature', label='PRISM 1990').opts(tools=['hover'])
# prism_1995_temp_mean  = prism_1995_ds.to(hv.Curve, kdims=['time'], vdims=['tmean'], group='Temperature', label='PRISM 1995').opts(tools=['hover'])
# prism_2003_temp_mean  = prism_2003_ds.to(hv.Curve, kdims=['time'], vdims=['tmean'], group='Temperature', label='PRISM 2003').opts(tools=['hover'])
# prism_2006_temp_mean  = prism_2006_ds.to(hv.Curve, kdims=['time'], vdims=['tmean'], group='Temperature', label='PRISM 2006').opts(tools=['hover'])

# wrf_1990_temp_mean = wrf_1990_ds_mean.to(hv.Curve, kdims=['time'], vdims=['T2C'], group='Temperature', label='WRF (ERA5) 1990').opts(tools=['hover'])
# wrf_1995_temp_mean = wrf_1995_ds_mean.to(hv.Curve, kdims=['time'], vdims=['T2C'], group='Temperature', label='WRF (ERA5) 1995').opts(tools=['hover'])
# wrf_2003_temp_mean = wrf_2003_ds_mean.to(hv.Curve, kdims=['time'], vdims=['T2C'], group='Temperature', label='WRF (ERA5) 2003').opts(tools=['hover'])
# wrf_2006_temp_mean = wrf_2006_ds_mean.to(hv.Curve, kdims=['time'], vdims=['T2C'], group='Temperature', label='WRF (ERA5) 2006').opts(tools=['hover'])

# pnnl_1990_temp_mean = pnnl_1990_ds_mean.to(hv.Curve, kdims=['time'], vdims=['T2C'], group='Temperature', label='PNNL Historical 1990').opts(tools=['hover'])
# pnnl_1995_temp_mean = pnnl_1995_ds_mean.to(hv.Curve, kdims=['time'], vdims=['T2C'], group='Temperature', label='PNNL Historical 1995').opts(tools=['hover'])
# pnnl_2003_temp_mean = pnnl_2003_ds_mean.to(hv.Curve, kdims=['time'], vdims=['T2C'], group='Temperature', label='PNNL Historical 2003').opts(tools=['hover'])
# pnnl_2006_temp_mean = pnnl_2006_ds_mean.to(hv.Curve, kdims=['time'], vdims=['T2C'], group='Temperature', label='PNNL Historical 2006').opts(tools=['hover'])

# mean_temp_1990 = (pnnl_1990_temp_mean * ornl_1990_temp_mean * prism_1990_temp_mean * wrf_1990_temp_mean * hv.VLine(dt.datetime.fromisoformat('1990-11-10')))\
#     .opts(legend_position='bottom_left', width=800, height=600)\
#     .redim.label(tmean='Temperature (°C)').redim.label(T2C='Temperature (°C)')\
#     .opts(title='Mean Temperature Comparison 1990')

# mean_temp_1995 = (pnnl_1995_temp_mean * ornl_1995_temp_mean * prism_1995_temp_mean * wrf_1995_temp_mean * hv.VLine(dt.datetime.fromisoformat('1995-11-29')))\
#     .opts(legend_position='bottom_left', width=800, height=600)\
#     .redim.label(tmean='Temperature (°C)').redim.label(T2C='Temperature (°C)')\
#     .opts(title='Mean Temperature Comparison 1995')

# mean_temp_2003 = (pnnl_2003_temp_mean * ornl_2003_temp_mean * prism_2003_temp_mean * wrf_2003_temp_mean * hv.VLine(dt.datetime.fromisoformat('2003-10-21')))\
#     .opts(legend_position='bottom_left', width=800, height=600)\
#     .redim.label(tmean='Temperature (°C)').redim.label(T2C='Temperature (°C)')\
#     .opts(title='Mean Temperature Comparison 2003')

# mean_temp_2006 = (pnnl_2006_temp_mean * ornl_2006_temp_mean * prism_2006_temp_mean * wrf_2006_temp_mean * hv.VLine(dt.datetime.fromisoformat('2006-11-06')))\
#     .opts(legend_position='bottom_left', width=800, height=600)\
#     .redim.label(tmean='Temperature (°C)').redim.label(T2C='Temperature (°C)')\
#     .opts(title='Mean Temperature Comparison 2006')

# hv.Layout([mean_temp_1990, mean_temp_1995, mean_temp_2003, mean_temp_2006]).opts(shared_axes=False).cols(2)



In [8]:
# Plot Snotel
snotel_2003 = xr.open_zarr('/data0/skagit_met/atmospheric_rivers/2003-10-14_2003-10-28_SNOTEL_hourly_data.zarr')
snotel_2006 = xr.open_zarr('/data0/skagit_met/atmospheric_rivers/2006-10-31_2006-11-13_SNOTEL_hourly_data.zarr')
snotel_1990 = xr.open_zarr('/data0/skagit_met/atmospheric_rivers/1990-11-03_1990-11-17_SNOTEL_daily_data.zarr')
snotel_1995 = xr.open_zarr('/data0/skagit_met/atmospheric_rivers/1995-11-22_1995-12-06_SNOTEL_hourly_data.zarr')
# snotel_sites
snotel_2003['T2C'] = (snotel_2003['AIR TEMP'] - 32) * (5/9)
s_T_03 = snotel_2003['T2C'].hvplot(x='time', by='site_name').redim.label(T2C='Temperature (°C)').opts(title="Temperature 2003 Event", width=800, height=600)

snotel_2006['T2C'] = (snotel_2006['AIR TEMP'] - 32) * (5/9)
s_T_06 = snotel_2006['T2C'].hvplot(x='time', by='site_name').redim.label(T2C='Temperature (°C)').opts(title="Temperature 2006 Event", width=800, height=600)

snotel_1990['T2C'] = (snotel_1990['AIR TEMP'] - 32) * (5/9)
s_T_90 = snotel_1990['T2C'].hvplot(x='time', by='site_name').redim.label(T2C='Temperature (°C)').opts(title="Temperature 1990 Event", width=800, height=600)

snotel_1995['T2C'] = (snotel_1995['AIR TEMP'] - 32) * (5/9)
s_T_95 = snotel_1995['T2C'].hvplot(x='time', by='site_name').redim.label(T2C='Temperature (°C)').opts(title="Temperature 1995 Event",width=800, height=600)

(s_T_90 + s_T_95 + s_T_03 + s_T_06).opts(shared_axes=False).cols(2)



:Layout
   .NdOverlay.I   :NdOverlay   [site_name]
      :Curve   [time]   (Temperature (°C))
   .NdOverlay.II  :NdOverlay   [site_name]
      :Curve   [time]   (Temperature (°C))
   .NdOverlay.III :NdOverlay   [site_name]
      :Curve   [time]   (Temperature (°C))
   .NdOverlay.IV  :NdOverlay   [site_name]
      :Curve   [time]   (Temperature (°C))

In [165]:
# elevs = skagit_dem.interp(x=pnnl_1990['CLONG'], y=pnnl_1990['CLAT'], method='nearest')
# pnnl_1990['elev'] = elevs
# pnnl_1990['elev'] = pnnl_1990.elev.drop_vars(['y', 'x'])
# wrf_1990
# print(skagit_dem.squeeze().drop_vars(['band', 'spatial_ref']))
# print(wrf_1990)
# Resample DEM to match ORNL grid resolution
dem_resampled = dem.interp(x=ornl_2006.lon, y=ornl_2006.lat, method='nearest')

# dem_resampled.max()
# Calculate average elevation per grid cell
avg_elevation = dem_resampled.mean(dim='band')

# Add the average elevation to the ORNL dataset
ornl_2006['elevation'] = (('lat', 'lon'), avg_elevation.values)
ornl_2006 = ornl_2006.set_coords('elevation')
end = ornl_2006.elevation.max().values
bins = np.arange(0, end, 500, dtype=int)
labels = [f'{bins[i-1]}-{bins[i]}' for i in range(1,len(bins))]
ornl_2006.groupby_bins("elevation", bins, labels=labels).mean(skipna=True).tmean.hvplot(x='time', by='elevation_bins').redim.label(elevation_bins='Elevation Band')



:NdOverlay   [elevation_bins]
   :Curve   [time]   (tmean)

In [173]:
dem_resampled = dem.interp(x=pnnl_2006.CLONG, y=pnnl_2006.CLAT, method='nearest')
avg_elevation = dem_resampled.mean(dim='band')

pnnl_2006['elevation'] = (('x', 'y'), avg_elevation.values)
pnnl_2006 = pnnl_2006.set_coords('elevation')
end = pnnl_2006.elevation.max().values
bins = np.arange(0, end, 500, dtype=int)
labels = [f'{bins[i-1]}-{bins[i]}' for i in range(1,len(bins))]
pnnl_2006_by_band = pnnl_2006.groupby_bins("elevation", bins, labels=labels).mean(skipna=True).T2C.hvplot(x='time', by='elevation_bins', title='PNNL 2006').redim.label(elevation_bins='Elevation Band')
wrf_2006_by_band = wrf_2006.set_coords("HGT").groupby_bins("HGT", bins, labels=labels).mean(skipna=True).T2C.hvplot(x='time', by='HGT_bins', title='WRF ERA 5 2006').redim.label(HGT_bins='Elevation Band')

pnnl_2006_by_band + wrf_2006_by_band


:Layout
   .NdOverlay.I  :NdOverlay   [elevation_bins]
      :Curve   [time]   (T2C)
   .NdOverlay.II :NdOverlay   [HGT_bins]
      :Curve   [time]   (T2C)

In [206]:
# pnnl_2006.T2C.hvplot.quadmesh(x='CLONG', y='CLAT', geo=True, dynamic=True, rasterize=True, project=True, ).opts(width=800, height=800, title='Temperature')

In [255]:
end = wrf_1990
bins = np.arange(0, end, 100, dtype=int)
labels = [f'{bins[i-1]}-{bins[i]}' for i in range(1,len(bins))]
elevation_wrf = wrf_1990.resample().groupby_bins("HGT", bins, labels=labels).mean(skipna=True).T2C.hvplot(x='time', by='HGT_bins').redim.label(HGT_bins='Elevation Band')
snotel_1990['elevation_m'] = snotel_1990['elevation_ft'] * 0.3048
label_dict = dict(zip([name for name in snotel_1990.site_name.values], [f'{name}:{int(elevation)} m' for name, elevation in zip(snotel_1990.site_name.values, snotel_1990.elevation_m.values)]))
snotel_elev = snotel_1990['T2C'].hvplot(x='time', by='site_name')
(snotel_elev*elevation_wrf).opts(width=800, height=600, title='WRF ERA5 1990 by 500 m Elevation Band vs SNOTEL', legend_labels=label_dict, gridstyle={'line_color': 'gray', 'line_width': 1, 'alpha': 0.7}, show_grid=True)

:Overlay
   .NdOverlay.I  :NdOverlay   [site_name]
      :Curve   [time]   (T2C)
   .NdOverlay.II :NdOverlay   [HGT_bins]
      :Curve   [time]   (T2C)

In [106]:
# import pint_xarray
from datetime import datetime as dt
ar_1990 = ('1990-11-03', '1990-11-17')
ar_1995 = ('1995-11-22', '1995-12-06')
ar_2003 = ('2003-10-14', '2003-10-28')
ar_2006 = ('2006-10-31', '2006-11-13')
ars = [ar_1990, ar_1995, ar_2003, ar_2006]
dem = xr.open_dataset('../Data/GIS/SkagitRiver_90mDEM.tif')
poly = gpd.read_file('../Data/GIS/SkagitRiver_BasinBoundary.json').geometry
skagit_dem = dem.rio.clip(poly)

# Remmeber WRF and PNNL are hourly
# SNOTEL and WRF alread have hgt/HGT data
def openDataset(dates: tuple[str], product: str, frequency: str = '') -> xr.Dataset:
    if product.lower() == 'pnnl':
        grid_path = '/data0/skagit_met/PNNL/historical/SERDP6km.geo_em.d01.nc'
        ds_grid = xr.open_dataset(grid_path).squeeze() # Drop Time=0 scalar dimension
        # Assign Coordinates in Xarray (keep landmask for viz) 
        dsg = ds_grid[['LANDMASK','CLONG','CLAT']]
        # Still not recognized as multidimensional coordinates
        # Rename to x and y To match data files
        dsg = dsg.set_coords(("CLONG", "CLAT")).rename(dict(south_north='y', west_east='x'))
        virtual_pnnl = '/data0/skagit_met/PNNL/historical/PNNL_historical.parquet'
        pnnl = xr.open_dataset(virtual_pnnl, engine='kerchunk', mask_and_scale=False)
        pnnl.assign_coords(dsg.coords)
        # Also bring in land mask
        pnnl.coords['LANDMASK'] = dsg.LANDMASK
        skagit = gpd.read_file('~/skagit-met/Data/GIS/SkagitBoundary.json')
        mask = vectorized.contains(skagit.geometry[0], pnnl.CLONG.values, pnnl.CLAT.values)
        ds = pnnl.where(mask)
        ds = ds.sel(time=slice(dates[0], dates[1]))
    elif product.lower() == 'ornl':
        start = dt.fromisoformat(dates[0]).year
        end = dt.fromisoformat(dates[1]).year
        ds = xr.open_zarr(f'/data0/skagit_met/atmospheric_rivers/{start}_{end}_ORNL_data.zarr')
        ds = ds.sel(time=slice(dates[0],dates[1]))
    else:
        ds = xr.open_zarr(f'/data0/skagit_met/atmospheric_rivers/{dates[0]}_{dates[1]}_{product}{"_"+frequency if frequency else ""}_data.zarr')

    if product in ['wrf_era5', 'pnnl']:
        ds['T2C'] = ds['T2'] - 273.15

    if product == 'SNOTEL':
        if 'AIR TEMP' in ds:
            ds['T2C'] = (ds['AIR TEMP'] - 32) * (5/9)

        if 'AVG AIR TEMP' in ds:
            ds['AVG_T2C'] = (ds['AVG AIR TEMP'] - 32) * (5/9)
        
        ds['elevation_m'] = ds['elevation_ft'] * 0.3048

    if product.lower() == 'wrf_era5':
        ds['PRCP'] = (ds['RAINC']+ ds['RAINNC'])
        ds = ds.set_coords('HGT')

    if product.lower() == 'ornl':
        ds['tmean'] = (ds.tmax + ds.tmin) / 2

    return ds

## Us CLAT for pnnnl x/y and x,y for index_x, index_y
def embedDEM(ds: xr.Dataset, dem: xr.Dataset, x: str, y: str, index_x: str = None, index_y: str = None, method: str  = 'nearest') -> xr.Dataset:
    dem_resampled = dem.interp(x=ds[x], y=ds[y], method=method)
    avg_elevation = dem_resampled.mean(dim='band')
    ds['elevation'] = ((index_y if index_y else y, index_x if index_x else x), avg_elevation.band_data.values)
    ds = ds.set_coords('elevation')
    return ds

def linePlotByElevationBand(ds: xr.Dataset, elev_var: str, bins, labels, var, title=''):
    return ds.groupby_bins(elev_var, bins, labels=labels).mean(skipna=True)[var].hvplot(x='time', by=f'{elev_var}_bins', title=title)

def linePlot(ds, var, snotel=False, x = None, y=None, label=''):
    if snotel: 
        label_dict = dict(zip([name for name in ds.site_name.values], [f'{name}:{int(elevation)} m' for name, elevation in zip(ds.site_name.values, ds.elevation_m.values)]))
        snotel_elev = ds[var].hvplot(x='time', by='site_name').opts(width=500, height=500)
        return snotel_elev
    else:
        return ds[var].mean(dim=[x,y], skipna=True).hvplot(x='time', label=label).opts(width=500, height=500)

def quadPlot(ds, var, x, y):
    return ds[var].hvplot.quadmesh(x=x, y=y, geo=True, dynamic=True, rasterize=True, project=True).opts(width=800, height=800)

In [107]:
pnnl_1990 = embedDEM(openDataset(ar_1990, 'pnnl'), skagit_dem, 'CLONG', 'CLAT', 'x', 'y')
pnnl_1995 = embedDEM(openDataset(ar_1995, 'pnnl'), skagit_dem, 'CLONG', 'CLAT', 'x', 'y')
pnnl_2003 = embedDEM(openDataset(ar_2003, 'pnnl'), skagit_dem, 'CLONG', 'CLAT', 'x', 'y')
pnnl_2006 = embedDEM(openDataset(ar_2006, 'pnnl'), skagit_dem, 'CLONG', 'CLAT', 'x', 'y')

ornl_1990 = embedDEM(openDataset(ar_1990, 'ORNL'), skagit_dem, 'lon', 'lat')
ornl_1995 = embedDEM(openDataset(ar_1995, 'ORNL'), skagit_dem, 'lon', 'lat')
ornl_2003 = embedDEM(openDataset(ar_2003, 'ORNL'), skagit_dem, 'lon', 'lat')
ornl_2006 = embedDEM(openDataset(ar_2006, 'ORNL'), skagit_dem, 'lon', 'lat')

prism_1990 = embedDEM(openDataset(ar_1990, 'PRISM'), skagit_dem, 'lon', 'lat')
prism_1995 = embedDEM(openDataset(ar_1995, 'PRISM'), skagit_dem, 'lon', 'lat')
prism_2003 = embedDEM(openDataset(ar_2003, 'PRISM'), skagit_dem, 'lon', 'lat')
prism_2006 = embedDEM(openDataset(ar_2006, 'PRISM'), skagit_dem, 'lon', 'lat')

wrf_1990 = openDataset(ar_1990, 'wrf_era5')
wrf_1995 = openDataset(ar_1995, 'wrf_era5')
wrf_2003 = openDataset(ar_2003, 'wrf_era5')
wrf_2006 = openDataset(ar_2006, 'wrf_era5')

snotel_1990 = openDataset(ar_1990, 'SNOTEL', 'daily')
snotel_1995 = openDataset(ar_1995, 'SNOTEL', 'daily')
snotel_2003 = openDataset(ar_2003, 'SNOTEL', 'hourly')
snotel_2006 = openDataset(ar_2006, 'SNOTEL', 'hourly')


In [113]:
label_dict = dict(zip([name for name in snotel_2006.site_name.values], [f'{name}:{int(elevation)} m' for name, elevation in zip(snotel_2006.site_name.values, snotel_2006.elevation_m.values)]))
snotel_2006.T2C.hvplot(x='time', by='site_name')\
    .opts(title='Snotel Temps in Skagit River Basin by Site', legend_labels=label_dict, show_grid=True, width=600, height=400)\
    .redim.label(T2C='Temperature (°C)')

:NdOverlay   [site_name]
   :Curve   [time]   (Temperature (°C))

In [114]:
snotel_2006

<xarray.Dataset> Size: 78kB
Dimensions:                    (time: 313, site: 6)
Coordinates:
    elevation_ft               (site) float64 48B dask.array<chunksize=(6,), meta=np.ndarray>
    lat                        (site) float64 48B dask.array<chunksize=(6,), meta=np.ndarray>
    lon                        (site) float64 48B dask.array<chunksize=(6,), meta=np.ndarray>
  * site                       (site) <U11 264B '711:WA:SNTL' ... '999:WA:SNTL'
    site_name                  (site) object 48B dask.array<chunksize=(6,), meta=np.ndarray>
  * time                       (time) datetime64[ns] 3kB 2006-10-31T08:00:00 ...
Data variables:
    ACCUMULATED PRECIPITATION  (time, site) float64 15kB dask.array<chunksize=(313, 6), meta=np.ndarray>
    AIR TEMP                   (time, site) float64 15kB dask.array<chunksize=(313, 6), meta=np.ndarray>
    SNOWDEPTH                  (time, site) float64 15kB dask.array<chunksize=(313, 6), meta=np.ndarray>
    SWE                        (time, site) float64 15kB dask.array<chunksize=(313, 6), meta=np.ndarray>
    T2C                        (time, site) float64 15kB dask.array<chunksize=(313, 6), meta=np.ndarray>
    elevation_m                (site) float64 48B dask.array<chunksize=(6,), meta=np.ndarray>

In [89]:
bin_end = 3000
bin_size = 500
bins = np.arange(0, bin_end, bin_size, dtype=int)
labels = [f'{bins[i-1]}-{bins[i]}' for i in range(1,len(bins))]

wrf_1990_daily = wrf_1990.resample(time='D').mean()
elevation_wrf = wrf_1990_daily.groupby_bins("HGT", bins, labels=labels).mean(skipna=True).T2C.hvplot(x='time', by='HGT_bins').redim.label(HGT_bins='Elevation Band')
# snotel_1990['elevation_m'] = snotel_1990['elevation_ft'] * 0.3048
label_dict = dict(zip([name for name in snotel_1990.site_name.values], [f'{name}:{int(elevation)} m' for name, elevation in zip(snotel_1990.site_name.values, snotel_1990.elevation_m.values)]))
snotel_elev = snotel_1990['T2C'].hvplot(x='time', by='site_name')
(snotel_elev*elevation_wrf).opts(width=800, height=600, title='WRF ERA5 1990 by 500 m Elevation Band vs SNOTEL', legend_labels=label_dict, gridstyle={'line_color': 'gray', 'line_width': 1, 'alpha': 0.7}, show_grid=True)

:Overlay
   .NdOverlay.I  :NdOverlay   [site_name]
      :Curve   [time]   (T2C)
   .NdOverlay.II :NdOverlay   [HGT_bins]
      :Curve   [time]   (T2C)

In [115]:
# snotel_1990_T2C = linePlot(snotel_1990, 'T2C', snotel=True)
# ornl_1990_T2C = linePlot(ornl_1990, 'tmean', x='lon', y='lat', label='ORNL 4Km Product')
# wrf_1990_T2C = linePlot(wrf_1990, 'T2C', x='x', y='y', label='WRF ERA5 9Km Product')
# prism_1990_T2C = linePlot(prism_1990, 'tmean', x='lon', y='lat', label='PRISM 4Km Product')
# pnnl_1990_T2C = linePlot(pnnl_1990, 'T2C',x='x', y='y', label='PNNL 6Km Product')

# snotel_1990['SWE'].hvplot(x='time', by='site_name') + (snotel_1990_T2C * ornl_1990_T2C * wrf_1990_T2C * prism_1990_T2C * pnnl_1990_T2C)\
#     .opts(title='Spatially Averaged Temperature 1990', show_grid=True, width=800, height=600)\
#     .redim.label(T2C='Temperature (°C)')\
#     .redim.label(tmean='Temperature (°C)')



In [102]:
# snotel_2006_T2C = linePlot(snotel_2006, 'T2C', snotel=True)
# ornl_2006_T2C = linePlot(ornl_2006, 'tmean', x='lon', y='lat', label='ORNL 4Km Product')
# wrf_2006_T2C = linePlot(wrf_2006, 'T2C', x='x', y='y', label='WRF ERA5 9Km Product')
# prism_2006_T2C = linePlot(prism_2006, 'tmean', x='lon', y='lat', label='PRISM 4Km Product')
# pnnl_2006_T2C = linePlot(pnnl_2006, 'T2C',x='x', y='y', label='PNNL 6Km Product')

# wrf_2006_daily = wrf_2006.resample(time="D").mean()
# wrf_2006_T2C_daily = linePlot(wrf_2006_daily, 'T2C',x='x', y='y', label='WRF ERA5 9Km Product Daily')



# (snotel_2006_T2C * ornl_2006_T2C * wrf_2006_T2C * prism_2006_T2C * pnnl_2006_T2C)\
#     .opts(title='Spatially Averaged Temperature 2006', show_grid=True, width=800, height=600)\
#     .redim.label(T2C='Temperature (°C)')\
#     .redim.label(tmean='Temperature (°C)')

# label_dict = dict(zip([name for name in snotel_2006.site_name.values], [f'{name}:{int(elevation)} m' for name, elevation in zip(snotel_2006.site_name.values, snotel_2006.elevation_m.values)]))
# w_binned = wrf_2006.groupby_bins("HGT", bins, labels=labels).mean(skipna=True).T2C.hvplot(x='time', by='HGT_bins').redim.label(HGT_bins='Elevation Band')
# (snotel_2006_T2C * w_binned)\
#     .opts(title='WRF ERA5 2005 Elevation Binned', show_grid=True, width=800, height=600, legend_labels=label_dict)\
#     .redim.label(T2C='Temperature (°C)')\
#     .redim.label(tmean='Temperature (°C)')


:Overlay
   .NdOverlay.I  :NdOverlay   [site_name]
      :Curve   [time]   (Temperature (°C))
   .NdOverlay.II :NdOverlay   [HGT_bins]
      :Curve   [time]   (Temperature (°C))

In [103]:
# bin_end = 3000
# bin_size = 500
# bins = np.arange(0, bin_end, bin_size, dtype=int)
# labels = [f'{bins[i-1]}-{bins[i]}' for i in range(1,len(bins))]

# # snotel_1990_T2C_bin = linePlot(snotel_1990, 'T2C', snotel=True)
# ornl_1990_T2C_bin = linePlotByElevationBand(ornl_1990, 'elevation', bins, labels, 'tmean', title='ORNL 4Km Product')
# wrf_1990_T2C_bin = linePlotByElevationBand(wrf_1990, 'HGT', bins, labels,'T2C', title='WRF ERA-5 9Km Product')
# prism_1990_T2C_bin = linePlotByElevationBand(prism_1990, 'elevation', bins, labels, 'tmean', title='Prism 4Km Product')
# pnnl_1990_T2C_bin = linePlotByElevationBand(pnnl_1990, 'elevation', bins, labels,'T2C', title='PNNL 6Km Product')

# snotel_1990_T2C * (ornl_1990_T2C_bin + wrf_1990_T2C_bin + prism_1990_T2C_bin + pnnl_1990_T2C_bin)\
#     .opts(title='Temperature by Elevation Band', width=800, height=600)\
#     .redim.label(T2C='Temperature (°C)')\
#     .redim.label(tmean='Temperature (°C)')\
#     .cols(2)      

<xarray.Dataset> Size: 2GB
Dimensions:      (time: 360, x: 450, y: 450)
Coordinates:
  * time         (time) datetime64[ns] 3kB 1990-11-03 ... 1990-11-17T23:00:00
    CLONG        (y, x) float32 810kB -126.7 -126.6 -126.5 ... -93.12 -93.03
    CLAT         (y, x) float32 810kB 28.93 28.94 28.94 ... 52.73 52.72 52.71
    LANDMASK     (y, x) float32 810kB 0.0 0.0 0.0 0.0 0.0 ... 1.0 1.0 1.0 1.0
    elevation    (y, x) float32 810kB nan nan nan nan nan ... nan nan nan nan
Dimensions without coordinates: x, y
Data variables:
    GLW          (time, x, y) float32 292MB dask.array<chunksize=(1, 450, 450), meta=np.ndarray>
    PREC_ACC_NC  (time, x, y) float32 292MB dask.array<chunksize=(1, 450, 450), meta=np.ndarray>
    Q2           (time, x, y) float32 292MB dask.array<chunksize=(1, 450, 450), meta=np.ndarray>
    SWDOWN       (time, x, y) float32 292MB dask.array<chunksize=(1, 450, 450), meta=np.ndarray>
    T2           (time, x, y) float32 292MB dask.array<chunksize=(1, 450, 450), meta=np.ndarray>
    U10          (time, x, y) float32 292MB dask.array<chunksize=(1, 450, 450), meta=np.ndarray>
    V10          (time, x, y) float32 292MB dask.array<chunksize=(1, 450, 450), meta=np.ndarray>
    T2C          (time, x, y) float32 292MB dask.array<chunksize=(1, 450, 450), meta=np.ndarray>
Attributes:
    CDI:          Climate Data Interface version 1.9.3 (http://mpimet.mpg.de/...
    CDO:          Climate Data Operators version 1.9.3 (http://mpimet.mpg.de/...
    Conventions:  CF-1.6
    author:       Xiaodong Chen (xiaodong.chen@pnnl.gov)
    script:       Constance:/pic/projects/hyperion/chen423/tools/archive_tool...
    tag:          HIST  CTRL